In [ ]:
import healpy as hp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.tests.helper import assert_quantity_allclose
import pickle
import sys
import os
from astropy import units as u

In [ ]:
import h5py
s4 = h5py.File("cmbs4_tophat.h5", mode="r")

In [ ]:
dataset = "s4_reference_design_noise_atmo_7splits"

In [ ]:
ch = f"LAT-LFL2_pole"

In [ ]:
if os.path.exists(f"plots/C_ell_{dataset}_{ch}.png"):
    sys.exit(0)

In [ ]:
sampling_frequency = dict(SAT=100*u.Hz, LAT=400*u.Hz)

In [ ]:
#telescope = "SAT"
#channel = "LFS1"
#site = "pole"

In [ ]:
telescope = ch.split("-")[0]
channel, site= ch.split("-")[1].split("_")

In [ ]:
C_ell = pickle.load(open(Path("output")/ dataset / f"C_ell_{telescope}.pkl", "rb"))

In [ ]:
telescope, channel, site

In [ ]:
C_ell.keys()

In [ ]:

nside = 4096 if telescope == "LAT" else 512

In [ ]:
from s4_design_sim_tool.core import base_folder

In [ ]:
%ls output/s4_reference_design_noise_atmo_7splits

In [ ]:
%ls output/s4_reference_design_noise_atmo_7splits/$ch

In [ ]:
import toml

config = toml.load(f"{dataset}.toml")

In [ ]:
hitmap = hp.read_map(Path("output") / dataset / ch / f"cmbs4_hitmap_{ch}_nside{nside}_1_of_1.fits")

In [ ]:
from s4_design_sim_tool.noise import get_tube_years
tube_years = get_tube_years(config, site, channel)
tube_years

In [ ]:
sampling_rate = hitmap.sum()
sampling_rate /= 365.25 * 24 * 3600
sampling_rate /= tube_years * config["experiment"]["observing_efficiency"]
sampling_rate /= s4[channel].attrs["detectors_per_tube"]
sampling_rate

In [ ]:
s4[channel].attrs["detectors_per_tube"]

In [ ]:
np.testing.assert_allclose(sampling_rate, sampling_frequency[telescope].value, rtol=1/100)

In [ ]:
log_hitmap = np.log10(hitmap)
log_hitmap[np.isinf(log_hitmap)] = hp.UNSEEN
hp.mollview(log_hitmap, unit="log10(samples)", title="Hitmap")
plt.savefig(f"plots/hitmap_{channel}_{site}_1_of_1.png")

In [ ]:
wcov = hp.read_map(Path("output") / dataset / ch / f"cmbs4_wcov_{ch}_nside{nside}_1_of_1.fits", (3))

In [ ]:
wcov.shape

In [ ]:
import astropy.units as u

In [ ]:
channel_NET = s4[channel].attrs["NET_uKsqrts"] * u.uK * u.s ** .5

In [ ]:
if telescope == "LAT" and site=="pole":
    channel_NET = s4[channel].attrs["pole_NET_uKsqrts"] * u.uK * u.s ** .5

In [ ]:
variance_QQ = wcov * u.K ** 2 * (hitmap / sampling_frequency[telescope]) 

In [ ]:
assert (variance_QQ.value == hp.UNSEEN).sum() == 0

In [ ]:
full_mission_NET = np.sqrt(np.median(variance_QQ[hitmap != 0]))

In [ ]:
full_mission_NET.to(channel_NET.unit)

In [ ]:
channel_NET

In [ ]:
(full_mission_NET/np.sqrt(2)).to(channel_NET.unit)

In [ ]:
assert_quantity_allclose(full_mission_NET, channel_NET*np.sqrt(2), rtol=10/100) 

In [ ]:
sky_fraction = (hitmap > 0).sum() / len(hitmap)

In [ ]:
sky_fraction

In [ ]:
pixarea = hp.nside2pixarea(nside)

In [ ]:
np.mean(wcov[hitmap > 0])* pixarea

In [ ]:
wcov [ wcov == hp.UNSEEN] = 0

## Atmosphere

In [ ]:
atmosphere_filename = Path(base_folder) / "00000000" / f"{site}_atmosphere_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits"

In [ ]:
input_atmophere = hp.ma(hp.read_map(atmosphere_filename, (0,1,2)))

In [ ]:
sqrt_hitmap = hitmap.copy()
sqrt_hitmap[hitmap>0] = np.sqrt(hitmap[hitmap>0])

In [ ]:
atmo_cl = \
hp.anafast(input_atmophere * sqrt_hitmap, use_pixel_weights=True) / np.mean(hitmap) / sky_fraction

In [ ]:
from s4_design_sim_tool.atmosphere import get_telecope_years

In [ ]:
telescope_years = get_telecope_years(config, site, channel)

In [ ]:
telescope_years

In [ ]:
from s4_design_sim_tool.core import simulations_observing_efficiency

In [ ]:
simulations_observing_efficiency

In [ ]:
atmo_cl *= 10 / 365.25
atmo_cl *= simulations_observing_efficiency[site.lower()].get(telescope, 1) / config["experiment"]["observing_efficiency"]                                                                    
atmo_cl /= telescope_years

## Noise

In [ ]:
noise_filename = Path(base_folder) / "00000000" / f"{site}_noise_{telescope}_{channel}_filtered_telescope_all_time_all_bmap.fits"

In [ ]:
#input_noise = hp.ma(hp.read_map(noise_filename, (0,1,2)))

In [ ]:
#noise_cl = hp.anafast(input_noise * sqrt_hitmap, use_pixel_weights=True) / np.mean(hitmap) / sky_fraction

In [ ]:
del sqrt_hitmap

In [ ]:
from s4_design_sim_tool.noise import get_thinfp
thinfp = get_thinfp(channel)

In [ ]:
thinfp

In [ ]:
#noise_cl *= 10 / 365.25
#noise_cl *= simulations_observing_efficiency[site.lower()].get(telescope, 1) / config["experiment"]["observing_efficiency"]                                                                    
#noise_cl /= tube_years
#noise_cl /= thinfp

In [ ]:
white_noise_Cl = channel_NET**2 \
/ (hitmap.sum() / sampling_frequency[telescope] / (hitmap>0).sum()) * pixarea / sky_fraction

In [ ]:
white_noise_Cl.decompose()

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")

In [ ]:
plt.figure(figsize=(8,5))

pols = ["TT", "EE", "BB"]

for i, pol in enumerate(pols):
    plt.loglog(C_ell[ch][i], alpha=.8, label=pol)

for i, pol in enumerate(pols):
    plt.loglog(atmo_cl[i], alpha=.8, ls="--", label=f"atmo {pol}")

#for i, pol in enumerate(pols):
#    plt.loglog(noise_cl[i], alpha=.8, ls="-.", label=f"noise {pol}")

plt.hlines(white_noise_Cl.to_value(u.K**2), 10, len(C_ell[ch][1]), ls=":",
           label="white noise T", color="black")
plt.hlines(white_noise_Cl.to_value(u.K**2)*2, 10, len(C_ell[ch][1]), ls=":",
           label="white noise P", color="grey")
plt.legend()
plt.title(f"Noise + Atmosphere simulations spectra for {ch}")
plt.ylabel("$C_\ell [K^2]$")
plt.xlabel("$\ell$")
plt.xlim((10, 3*nside))
plt.grid(True)
plt.savefig(f"plots/C_ell_{dataset}_{ch}.png");